### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [2]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [3]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [5]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [6]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-43    3.8   33     0.15   1        Bom      0.99      
-78    4.8   3      0.23   1        Bom      0.77      
-59    1.5   28     0.21   1        Bom      1.00      
-82    7.1   15     0.83   0        Ruim     1.00      
-68    9.7   37     0.72   0        Ruim     1.00      
-64    8.2   64     0.86   0        Ruim     1.00      
-82    5.7   36     0.83   0        Ruim     1.00      
-64    6.7   98     0.78   0        Ruim     1.00      
-50    1.1   9      0.29   1        Bom      1.00      
-46    2.1   33     0.39   1        Bom      0.99      
-64    8.5   98     0.63   0        Ruim     1.00      
-40    2.7   30     0.10   1        Bom      1.00      
-44    3.3   10     0.38   1        Bom      0.99      
-48    2.6   18     0.18   1        Bom      1.00      
-55    1.0   15     0.35   1        Bom      1.00      
-39    2.8   11     0.34   1        Bom      1.0

### Save

In [7]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
